In [ ]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
import os

# （可选）Binance API Key（可以为空）
api_key = ''
api_secret = ''

client = Client(api_key, api_secret)

# === 配置参数 ===
symbol = 'SHIBUSDT'  # Shiba Inu 的交易对
interval = Client.KLINE_INTERVAL_1HOUR

# 设置开始和结束时间：从 2024-05-03 开始，抓取 960 小时（40 天）
start_time = datetime(2024, 10, 10)
end_time = start_time + timedelta(hours=960)
start_str = start_time.strftime('%Y-%m-%d %H:%M:%S')
end_str = end_time.strftime('%Y-%m-%d %H:%M:%S')

# 获取 K 线数据（最多 1000 条，抓 960 就够）
klines = client.get_historical_klines(
    symbol=symbol,
    interval=interval,
    start_str=start_str,
    end_str=end_str,
    limit=960
)

# 转换成 DataFrame
df = pd.DataFrame(klines, columns=[
    "timestamp", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base", "taker_buy_quote", "ignore"
])

# 处理时间戳 & 保留主列
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
df = df[["timestamp", "open", "high", "low", "close", "volume"]]
df.set_index("timestamp", inplace=True)

# 获取最后一条记录的日期（UTC）
last_date = df.index[-1].strftime("%Y%m%d")

# 创建保存目录
save_dir = "Shiba_Price"
os.makedirs(save_dir, exist_ok=True)

# 文件路径和文件名
file_name = f"shiba_hourly_ohlcv_until_{last_date}.csv"
file_path = os.path.join(save_dir, file_name)

# 保存为 CSV 文件
df.to_csv(file_path)
print(f"✅ 数据保存至: {file_path}")
print(df.tail())


✅ 数据保存至: Shibainu_Price/shiba_hourly_ohlcv_until_20241118.csv
                           open        high         low       close  \
timestamp                                                             
2024-11-18 19:00:00  0.00002464  0.00002472  0.00002402  0.00002437   
2024-11-18 20:00:00  0.00002438  0.00002479  0.00002422  0.00002452   
2024-11-18 21:00:00  0.00002452  0.00002484  0.00002447  0.00002458   
2024-11-18 22:00:00  0.00002458  0.00002488  0.00002453  0.00002477   
2024-11-18 23:00:00  0.00002478  0.00002488  0.00002447  0.00002486   

                              volume  
timestamp                             
2024-11-18 19:00:00  575063152234.00  
2024-11-18 20:00:00  513878456353.00  
2024-11-18 21:00:00  346470953819.00  
2024-11-18 22:00:00  174560816134.00  
2024-11-18 23:00:00  202715484001.00  
